In [ ]:
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import datetime
import os

In [ ]:
date_data = []
comments_data = []
rating_data = []
result_data = []

for n in range(1, 100):
  time.sleep(1)
  url = f'https://autoc-one.jp/ullo/shopReview/pa{n}/'
  res = requests.get(url)
  soup = BeautifulSoup(res.text,"html.parser")

  # 日付の取得
  date = soup.find_all("span",attrs={"shop-review__user-shop"})
  for i in range(len(date)):
    d = date[i].text[-11:]
    d = d.replace("年","-").replace("月","-").replace("日","")
    date_data.append(d)


  # 評価の取得
  rating = soup.find_all("span",attrs={"shop-review-total-rating__value"})
  for i in range(len(rating)):
    rating_data.append(float(rating[i].text))

  # コメントの取得
  comments  = soup.select(".shop-review__secondary")
  for i in range(len(comments)):
    m = len(comments[i].find_all("p"))
    cms = ""
    for c in range(m):
      cms += comments[i].find_all("p")[c].text.replace("         ","").replace("\n","").replace("\r","").replace(" ","")
    comments_data.append(cms)

  #　買取が成立したかどうか
  s = soup.select(".shop-review__item")
  for i in range(len(s)):
    if "買取成立" in s[i].text:
      result_data.append(1)
    else:
      result_data.append(0)


In [ ]:
data = {"date":1,
        "rate":1,
        "comment":1,
        "result":1}

data["date"] = date_data
data["rate"] = rating_data
data["comment"] = comments_data
data["result"] = result_data
df = pd.DataFrame(data)

In [ ]:
df.head(10)

In [ ]:
df["date"] = pd.to_datetime(df["date"])
df.sort_values(by = 'date', ascending = True, inplace = True)
df.head()

In [ ]:
df.to_csv("trade_in_appraisal.csv")